# Comparador COVID19 México
## Herramienta para comparar fácilmente la cantidad de casos y decesos de COVID19 de dos fechas distintas
Esta herramienta tiene como propósito obtener los casos y decesos ocurridos entre dos fechas especificadas a causa de COVID19. La información es obtenida directamente de los [Datos Abiertos de la Dirección General de Epidemiología del Gobierno de México](https://www.gob.mx/salud/documentos/datos-abiertos-152127), y es una alternativa a realizar el proceso de forma manual (tarea que no es simple, puesto que la base de datos cuenta con más de tres millones de registros, lo que excede la capacidad de programas comúnes como Excel).

Utilizar la herramienta es bastante simple:
- Llenar los datos del formulario a continuación. Se puede presionar el ícono del calendario para hacerlo de forma más sencilla. Hay que tomar en cuenta que las fechas están en formato año/mes/día. De igual manera, la base de datos se actualiza cada día al rededor de las 20:00, hora de la CDMX. Si se ingresa una fecha que aún no está disponible, el comparador tomará la  el último registro disponible.
- Presionar el ícono de "reproducir" que está del lado izquierdo. El proceso puede tomar unos minutos puesto que tiene que descargar la información de la base de datos. 
- Cuando el proceso anterior ha terminado, hay que reproducir las celdas de la sección **Resultados del comparador**. Si se quiere descargar las tablas producidas, se tiene que reproducir la última celda.

Para dudas o comentarios, estoy en Twitter como [@ArrobaRicardoGE](https://twitter.com/ArrobaRicardoGE). 

Este proyecto es de código abierto. Para contribuir, se puede visitar el respositorio en https://github.com/ArrobaRicardoGE/comparador-covid-mexico.

# Comparador

In [ ]:
'''
ATENCION: Modificar el codigo podria afectar el funcionamiento de la herramienta,
          hacerlo bajo su propio riesgo. El repositorio del proyecto esta dispo-
          nible en https://github.com/ArrobaRicardoGE/comparador-covid-mexico.
'''

state_ids = {
  'AGUASCALIENTES':1,
  'BAJA CALIFORNIA':2,
  'BAJA CALIFORNIA SUR':3,
  'CAMPECHE':4,
  'COAHUILA DE ZARAGOZA':5,
  'COLIMA':6,
  'CHIAPAS':7,
  'CHIHUAHUA':8,
  'CIUDAD DE MEXICO':9,
  'DURANGO':10,
  'GUANAJUATO':11,
  'GUERRERO':12,
  'HIDALGO':13,
  'JALISCO':14,
  'MEXICO':15,
  'MICHOACAN DE OCAMPO':16,
  'MORELOS':17,
  'NAYARIT':18,
  'NUEVO LEON':19,
  'OAXACA':20,
  'PUEBLA':21,
  'QUERETARO':22,
  'QUINTANA ROO':23,
  'SAN LUIS POTOSI':24,
  'SINALOA':25,
  'SONORA':26,
  'TABASCO':27,
  'TAMAULIPAS':28,
  'TLAXCALA':29,
  'VERACRUZ DE IGNACIO DE LA LLAVE':30,
  'YUCATAN':31,
  'ZACATECAS':32,
  'ESTADOS UNIDOS MEXICANOS':36,
  'NO APLICA':97,
  'SE IGNORA':98,
  'NO ESPECIFICADO':99,
}

#@markdown Fecha reciente
recent_date = '2021-01-04' #@param {type: 'date'}
#@markdown Fecha anterior
past_date = '2021-01-03' #@param {type: 'date'}
#@markdown Entidad federativa
state = 'AGUASCALIENTES' #@param type: 'dropdown', ['AGUASCALIENTES','BAJA CALIFORNIA','BAJA CALIFORNIA SUR','CAMPECHE','COAHUILA DE ZARAGOZA','COLIMA','CHIAPAS','CHIHUAHUA','CIUDAD DE MEXICO','DURANGO','GUANAJUATO','GUERRERO','HIDALGO','JALISCO','MEXICO','MICHOACAN DE OCAMPO','MORELOS','NAYARIT','NUEVO LEON','OAXACA','PUEBLA','QUERETARO','QUINTANA ROO','SAN LUIS POTOSI','SINALOA','SONORA','TABASCO','TAMAULIPAS','TLAXCALA','VERACRUZ DE IGNACIO DE LA LLAVE','YUCATAN','ZACATECAS','ESTADOS UNIDOS MEXICANOS','NO APLICA','SE IGNORA','NO ESPECIFICADO']

state = state_ids[state]

import pandas as pd

def read_file(date):
    try:
        y,m,d = date.split('-')
        if y == '2021':
            try:
                return pd.read_csv(f'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2021/{m}/datos_abiertos_covid19_{d}.{m}.2021.zip',compression='zip',low_memory=False,encoding='ISO-8859-1')
            except:
                return pd.read_csv('http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip',compression='zip',low_memory=False,encoding='ISO-8859-1')
        else:
            return pd.read_csv(f'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/{m}/datos_abiertos_covid19_{d}.{m}.2020.zip',compression='zip',low_memory=False,encoding='ISO-8859-1')
    except Exception as e:
        print(str(e) + '\nLa fecha especificada no fue encontrada en la base de datos, o ocurrió un error')

try:
  print(f'Descargando y leyendo primer archivo ({recent_date})...')
  recent_d = read_file(recent_date)
  print(f'Descargando y leyendo segundo archivo ({past_date})...')
  past_d = read_file(past_date)

  print('Filtrando los datos...')
  recent_loc = recent_d[recent_d.ENTIDAD_UM == state]
  past_loc = past_d[past_d.ENTIDAD_UM == state]

  recent_dt = recent_loc[recent_loc.FECHA_DEF != '9999-99-99']
  past_dt = past_loc[past_loc.FECHA_DEF != '9999-99-99']
  print('¡Finalizado!')
except:
    print('Ha ocurrido un error, volver a intentarlo.')

# Resultados del comparador

In [ ]:
#@title ## Nuevos casos entre las dos fechas proporcionadas 
new_reg = pd.concat([recent_loc,past_loc]).drop_duplicates(keep=False,subset=['ID_REGISTRO'])
new_reg

In [ ]:
#@title ## Nuevos decesos entre las dos fechas proporcionadas
new_dts = pd.concat([recent_dt,past_dt]).drop_duplicates(keep=False,subset=['ID_REGISTRO'])
new_dts

In [ ]:
#@title ## Ejecutar en caso de que se quiera descargar la tabla de **Nuevos Casos**
from google.colab import files

new_reg.to_csv(f'NuevosCasos_{past_date}_{recent_date}.csv.gz',compression='gzip',index=False)

files.download(f'NuevosCasos_{past_date}_{recent_date}.csv.gz')

In [ ]:
#@title ## Ejecutar en caso de que se quiera descargar la tabla de **Nuevos Decesos**
from google.colab import files

new_dts.to_csv(f'NuevosDecesos_{past_date}_{recent_date}.csv.gz',compression='gzip',index=False)

files.download(f'NuevosDecesos_{past_date}_{recent_date}.csv.gz')